In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import RootMeanSquaredError
from time import time
from sklearn.metrics import mean_absolute_error
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [76]:
def data():
    training = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/training_preprocessed")
    validation = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/validation_preprocessed")
    holdout = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/holdout_preprocessed")
    holdout_predictions_df = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/holdout_preds_preprocessed")
    test = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/test_preprocessed")
    
    X_train = training.drop(columns="SalePrice")
    y_train = training["SalePrice"]
    X_valid = validation.drop(columns="SalePrice")
    y_valid = validation["SalePrice"]
    X_holdout = holdout.drop(columns="Actual_SalePrice")
    y_holdout = holdout["Actual_SalePrice"]
    X_test = test
    holdout_predictions_df = holdout_predictions_df
    return X_train, y_train, X_valid, y_valid, X_holdout, y_holdout, X_test, holdout_predictions_df

In [77]:
X_train, y_train, X_valid, y_valid, X_holdout, y_holdout, X_test, holdout_predictions_df = data()

In [104]:
# Since this model uses k-fold validation, we don't need separate training and validation datasets
X_train = X_train.append(X_valid).reset_index().drop(columns="index")
y_train = y_train.append(y_valid).reset_index().drop(columns="index").values

<ipython-input-104-33faa622c081>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(X_valid).reset_index().drop(columns="index")
<ipython-input-104-33faa622c081>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(y_valid).reset_index().drop(columns="index").values


In [108]:
wandb.init(project="house-price-prediction", entity="luiscostigan")

wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [109]:
def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(wandb.config.dropout1))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss="mse", metrics=[RootMeanSquaredError()])
    
    return model

In [110]:
from sklearn.model_selection import KFold

# Instantiate the validator, have it spit out the indices, generate batches from the indices, and feed those batches to the model.
kf = KFold(n_splits=5)
kf.get_n_splits(X_train, y_train)

5

Currently trying to implement RepeatedKFold cross validation with the Keras Sequential model, while recording model results on WandB.

Sometimes I am getting a failed-1 error from WandB.
    - 

In [112]:
from wandb.keras import WandbCallback

sweep_config = {
  "name": "keras-sequential-model-sweep",
  "method": "random",
  "parameters": {
    "dropout1": {
      "min": 0.0,
      "max": 0.6
    },
    "epochs": {
      "values": [30,50,100,200]
    },
    "batch_size": {
      "values": [16, 64, 128, 512]
    }
  }
}

config_defaults = {
  "dropout1": 0.5,
  "epochs": 50,
  "batch_size": 64
}

def train():

  rmse_per_fold = []
  loss_per_fold = []
  fold_no = 1

  # Go through each split, and get the index number for each (currently 50, 10 x 5 repeated k-fold) 
  for train, test in kf.split(X_train, y_train):

    # With the current session in WandB
    with wandb.init(config=config_defaults) as run:

      # Recreate the model each time for each new batch
      model = None # Not sure if this step is necessary
      model = create_model()

      # Fit model on new batches
      model.fit(
        np.asarray(X_train), 
        y_train, 
        epochs=wandb.config.epochs, 
        batch_size=wandb.config.batch_size, 
        verbose=0,
        callbacks=[WandbCallback()], 
        validation_data=(np.asarray(X_train),y_train)
        )
      
      # Generate data for each
      scores = model.evaluate(np.asarray(X_train), y_train, callbacks=[WandbCallback()])
      print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}')
      rmse_per_fold.append(scores[1])
      loss_per_fold.append(scores[0])

      # Increase fold number
      fold_no = fold_no + 1

      # == Provide average scores ==
      print('------------------------------------------------------------------------')
      print('Score per fold')
      for i in range(0, len(rmse_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - RMSE: {rmse_per_fold[i]}%')
      print('------------------------------------------------------------------------')
      print('Average scores for all folds:')
      print(f'> RMSE: {np.mean(rmse_per_fold)} (+- {np.std(rmse_per_fold)})')
      print(f'> Loss: {np.mean(loss_per_fold)}')
      print('------------------------------------------------------------------------')

      wandb.log({
        "Mean Validation RMSE (all folds)": np.mean(rmse_per_fold),
        "Mean Validation Loss (all folds)": np.mean(loss_per_fold) 
        })

keras_sequential_sweep_1 = wandb.sweep(sweep_config, project="house-price-prediction", entity="luiscostigan")

count = 10
wandb.agent(keras_sequential_sweep_1, function=train, count=count)

Create sweep with ID: ru0ykkjf
Sweep URL: https://wandb.ai/luiscostigan/house-price-prediction/sweeps/ru0ykkjf


wandb: Agent Starting Run: iung0a55 with config:
wandb: 	batch_size: 16
wandb: 	dropout1: 0.3838429917661869
wandb: 	epochs: 50
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


37/37 [==============================] - 0s 3ms/step - loss: 642050368.0000 - root_mean_squared_error: 25338.7129
Score for fold 1: loss of 642050368.0; root_mean_squared_error of 25338.712890625
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 642050368.0 - RMSE: 25338.712890625%
------------------------------------------------------------------------
Average scores for all folds:
> RMSE: 25338.712890625 (+- 0.0)
> Loss: 642050368.0
------------------------------------------------------------------------


Mean Validation Loss (all folds),▁
Mean Validation RMSE (all folds),▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_root_mean_squared_error,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Mean Validation Loss (all folds),642050368.0
Mean Validation RMSE (all folds),25338.71289
best_epoch,49
best_val_loss,642050496.0


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

37/37 [==============================] - 0s 1ms/step - loss: 701038464.0000 - root_mean_squared_error: 26477.1309
Score for fold 4: loss of 701038464.0; root_mean_squared_error of 26477.130859375
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 689857664.0 - RMSE: 26265.142578125%
------------------------------------------------------------------------
> Fold 2 - Loss: 696049856.0 - RMSE: 26382.755859375%
------------------------------------------------------------------------
> Fold 3 - Loss: 772503680.0 - RMSE: 27793.951171875%
------------------------------------------------------------------------
> Fold 4 - Loss: 701038464.0 - RMSE: 26477.130859375%
------------------------------------------------------------------------
Average scores for all folds:
> RMSE: 26729.7451171875 (+- 618.9922363051668)
> Loss: 714862416.0
-------------------------------------

Mean Validation Loss (all folds),▁
Mean Validation RMSE (all folds),▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▆▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,██▇▅▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_root_mean_squared_error,█▇▆▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Mean Validation Loss (all folds),714862416.0
Mean Validation RMSE (all folds),26729.74512
best_epoch,48
best_val_loss,692058112.0


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


37/37 [==============================] - 0s 1ms/step - loss: 692565696.0000 - root_mean_squared_error: 26316.6426
Score for fold 5: loss of 692565696.0; root_mean_squared_error of 26316.642578125
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 689857664.0 - RMSE: 26265.142578125%
------------------------------------------------------------------------
> Fold 2 - Loss: 696049856.0 - RMSE: 26382.755859375%
------------------------------------------------------------------------
> Fold 3 - Loss: 772503680.0 - RMSE: 27793.951171875%
------------------------------------------------------------------------
> Fold 4 - Loss: 701038464.0 - RMSE: 26477.130859375%
------------------------------------------------------------------------
> Fold 5 - Loss: 692565696.0 - RMSE: 26316.642578125%
------------------------------------------------------------------------
Average 

Mean Validation Loss (all folds),▁
Mean Validation RMSE (all folds),▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▆▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,██▇▅▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_root_mean_squared_error,█▇▆▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Mean Validation Loss (all folds),710403072.0
Mean Validation RMSE (all folds),26647.12461
best_epoch,48
best_val_loss,692033920.0


In [113]:
# Enter best params from sweep
best_params = {
    "dropout": 0.1589,
    "epochs": 50,
    "batch_size": 16
}

def make_predictions(best_params, dataset):
    
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(best_params.get("dropout")))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss="mse", metrics=[RootMeanSquaredError()])
              
    model.fit(X_train, y_train, epochs=best_params.get("epochs"),
              batch_size=best_params.get("batch_size"), verbose = 0)
    
    preds  = model.predict(dataset, best_params.get("batch_size"), verbose = 0)
              
    return preds

In [114]:
test_predictions = make_predictions(best_params, X_test)
holdout_predictions = make_predictions(best_params, X_holdout)

## Create Submissions

In [115]:
# Generating submission for test dataset
d = {"Id":X_test.index,"SalePrice":test_predictions.flatten()}
submission = pd.DataFrame(data=d, index=None)

submission["Id"] = submission["Id"] + 1461

submission.to_csv("submission_nn.csv",index=False)

## Blend Predictions

First, try taking the mean of predictions from the decision tree algorithm and neural net algorithm.

In [118]:
# Read CSVs
decision_tree_predictions = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/submission_dt.csv")
neural_net_predictions = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/tensorflow/submission_nn.csv")

# Merge CSVs on Id column
decision_tree_predictions["neural_net_predictions"] = neural_net_predictions["SalePrice"]

# Rename decision tree predictions column
decision_tree_predictions = decision_tree_predictions.rename(columns={"SalePrice":"decision_tree_predictions"})

# Add new column with mean
decision_tree_predictions["SalePrice"] = decision_tree_predictions[["neural_net_predictions","decision_tree_predictions"]].mean(axis=1)

# Drop other columns
decision_tree_predictions = decision_tree_predictions[["Id","SalePrice"]]

mean_predictions = decision_tree_predictions

In [119]:
mean_predictions.to_csv("submission_mean.csv",index=False)

Next, try defining a meta-model to best blend predictions.

In [ ]:
# You currently have the following problems:
# 1) You have trained and validated using k-fold validation on a training set that was already split into a validation set.
# 2) You need to keep a "hold-out" data set to validate merged predictions, that was not used at all in the DT and NN models.

In [88]:
holdout_predictions_df["NN_predictions"] = holdout_predictions
holdout_predictions_df = holdout_predictions_df[["DT_predictions", "NN_predictions", "Actual_SalePrice"]]

In [ ]:
# Use a super-simple Linear Regression to generate blended predictions

In [155]:
test_predictions_dt = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/submission_dt.csv")
test_predictions_nn = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/tensorflow/submission_nn.csv")

In [156]:
# Merge CSVs on Id column
test_predictions_dt["NN_predictions"] = test_predictions_nn["SalePrice"]

# Rename decision tree predictions column
test_predictions_dt = test_predictions_dt.rename(columns={"SalePrice":"DT_predictions"})

# Rename df
blended_predictions_df = test_predictions_dt

In [157]:
blended_predictions_df

,Id,DT_predictions,NN_predictions
0,1461,127293.890818,115650.310
1,1462,161564.270609,163336.770
2,1463,181414.391385,180642.690
3,1464,191273.208250,191659.220
4,1465,184825.733316,197580.620
...,...,...,...
1454,2915,79642.883442,79426.590
1455,2916,85594.040995,74407.945
1456,2917,153089.149513,165210.940
1457,2918,113378.264361,110647.040


In [192]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, mean_squared_error
import xgboost as xgb
import lightgbm as lgb


rmse = make_scorer(mean_squared_error, squared=False)

X = holdout_predictions_df[["DT_predictions", "NN_predictions"]]
y = holdout_predictions_df["Actual_SalePrice"]

#folds = KFold(n_splits=5, shuffle=True, random_state = 100)

estimators = [
    {
        "clf": (LinearRegression(),)
    },
    {
        "clf": (Ridge(),)
    },
    {
        "clf": (xgb.XGBRegressor(),)
    },
    {
       "clf": (lgb.LGBMRegressor(),)
    }
]

pipe = Pipeline([("clf", LinearRegression())])

grid_search = GridSearchCV(pipe, estimators, cv=10, scoring=rmse)
grid_search.fit(X,y)

/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pa

GridSearchCV(cv=10, estimator=Pipeline(steps=[('clf', LinearRegression())]),
             param_grid=[{'clf': (LinearRegression(),)}, {'clf': (Ridge(),)},
                         {'clf': (XGBRegressor(base_score=None, booster=None,
                                               colsample_bylevel=None,
                                               colsample_bynode=None,
                                               colsample_bytree=None,
                                               enable_categorical=False,
                                               gamma=None, gpu_id=None,
                                               importance_type=None,
                                               interaction_constraints=None,...
                                               min_child_weight=None,
                                               missing=nan,
                                               monotone_constraints=None,
                                               n_estimator

In [193]:
grid_search.best_estimator_

Pipeline(steps=[('clf',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, enable_categorical=False,
                              gamma=0, gpu_id=-1, importance_type=None,
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=16, num_parallel_tree=1, predictor='auto',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

In [194]:
grid_search_blended_predictions = grid_search.predict(blended_predictions_df[["DT_predictions", "NN_predictions"]])
grid_search_blended_predictions

/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


array([125867.625, 167306.84 , 177933.84 , ..., 149429.88 , 111527.47 ,
       228091.75 ], dtype=float32)

In [181]:
grid_search_blended_predictions_df = blended_predictions_df
grid_search_blended_predictions_df["SalePrice"] = grid_search_blended_predictions
grid_search_blended_predictions_df = grid_search_blended_predictions_df.drop(columns=["DT_predictions","NN_predictions"])
grid_search_blended_predictions_df.to_csv("submission_gridsearch_blended.csv",index=False)